In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import trange

In [2]:
# --- 1️⃣ Data Loading ---
transform = transforms.Compose([
    transforms.ToTensor(),                     # Convert to Tensor
    transforms.Normalize((0.1307,), (0.3081,)) # Normalize MNIST
])

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=512, shuffle=False)

In [3]:
# --- 2️⃣ Define Model ---
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)   # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
# --- 3️⃣ Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [5]:
# --- 4️⃣ Training Loop ---
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # --- Evaluation ---
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            preds = model(X_test).argmax(dim=1)
            correct += (preds == y_test).sum().item()
            total += y_test.size(0)

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(train_loader):.4f} - Test Acc: {acc:.2f}%")

Epoch 1/5 - Loss: 0.3432 - Test Acc: 95.96%
Epoch 2/5 - Loss: 0.1155 - Test Acc: 97.06%
Epoch 3/5 - Loss: 0.0714 - Test Acc: 97.61%
Epoch 4/5 - Loss: 0.0490 - Test Acc: 97.83%
Epoch 5/5 - Loss: 0.0397 - Test Acc: 97.68%


In [ ]:
# --- 5️⃣ Save model for web use (ONNX) ---
import torch, onnx, os

# make sure both model and dummy input are on CPU
model = model.to("cpu").eval()
dummy_input = torch.randn(1, 1, 28, 28, device="cpu")

onnx_path = "mnist_model_nn1.onnx"

torch.onnx.export(
    model,
    (dummy_input,),
    onnx_path,
    opset_version=18,
    export_params=True,
    do_constant_folding=True,
    verbose=False,
)

print(f"✅ Exported single-file ONNX at {onnx_path}, size ≈ {os.path.getsize(onnx_path)/1e6:.2f} MB")

# optional verification
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)
print("✅ Model check passed")



✅ Exported single-file ONNX at mnist_model_nn1.onnx, size ≈ 0.01 MB
✅ Model check passed


In [39]:
# Merge ONNX files with external data into a single file
import onnx, os
from onnx.external_data_helper import load_external_data_for_model

# 1️⃣ Point to the .onnx file (and ensure the .onnx.data file is in the same folder)
merged = "single_" + onnx_path

# 2️⃣ Load and embed the external data
model = onnx.load(onnx_path)
load_external_data_for_model(model, os.path.dirname(onnx_path))

# 3️⃣ Save a self-contained file
onnx.save(model, merged)

print("✅ Merged single-file ONNX:", merged, f"({os.path.getsize(merged)/1e6:.2f} MB)")


✅ Merged single-file ONNX: single_mnist_model_nn1.onnx (2.69 MB)
